# Portada.

### P/E estimator.

In [56]:
### Libraries to use.
import pandas as pd 
import numpy as np
import yfinance as yf
import yahoo_fin.stock_info as si
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

First, let's get the tickers of the companies we're interested in. [Click here to get them.](https://www.nasdaq.com/market-activity/stocks/screener)

In [11]:
### Import tickers.
tickers = pd.read_csv('Large_cap_growth_tickers.csv')

### Filter tickers with at least 5 years since IPO.
tickers['IPO Dif'] = datetime.datetime.now().year - tickers['IPO Year']
tickers = tickers[(tickers['IPO Dif'] > 5)]
tickers.head(10)

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,IPO Dif
5,AKAM,Akamai Technologies Inc. Common Stock,$116.96,-0.66,-0.561%,1.906351e+10,United States,1999.0,1779685,Miscellaneous,Business Services,22.0
6,ALGN,Align Technology Inc. Common Stock,$627.92,8.27,1.335%,4.969144e+10,United States,2001.0,356263,Health Care,Industrial Specialties,20.0
7,ALNY,Alnylam Pharmaceuticals Inc. Common Stock,$181.81,1.72,0.955%,2.137086e+10,United States,2004.0,765310,Health Care,Major Pharmaceuticals,17.0
8,AMAT,Applied Materials Inc. Common Stock,$137.18,-0.82,-0.594%,1.253799e+11,United States,1972.0,5528852,Technology,Semiconductors,49.0
10,AMGN,Amgen Inc. Common Stock,$244.74,-0.11,-0.045%,1.406163e+11,United States,1983.0,1372060,Health Care,Major Pharmaceuticals,38.0
11,ANSS,ANSYS Inc. Common Stock,$358.23,3.85,1.086%,3.122031e+10,United States,1996.0,186296,Technology,Computer Software: Prepackaged Software,25.0
14,AVGO,Broadcom Inc. Common Stock,$476.33,-2.69,-0.562%,1.954194e+11,United States,2009.0,1211690,Technology,Semiconductors,12.0
16,BMRN,BioMarin Pharmaceutical Inc. Common Stock,$78.46,-1.19,-1.494%,1.433798e+10,United States,1999.0,593326,Health Care,Major Pharmaceuticals,22.0
19,CDW,CDW Corporation Common Stock,$177.46,0.21,0.118%,2.488052e+10,United States,2013.0,457815,Technology,EDP Services,8.0
20,CERN,Cerner Corporation Common Stock,$78.06,0.37,0.476%,2.352081e+10,United States,1986.0,1657073,Technology,EDP Services,35.0


In [46]:
### Make a list of all the available tickers.
tickers_list = tickers['Symbol'].tolist()

### Get the P/E ratio of the stocks.
PE_vector = {i: si.get_quote_table(i)['PE Ratio (TTM)'] for i in tickers_list}
PE_values = pd.DataFrame.from_dict(PE_vector, orient='index', columns=['P/E ratio'])
PE_values.index.name = 'Stocks'

PE_values.dropna(inplace=True)
PE_values

In [51]:
### Sorting values. Top 30 stocks with the lowest P/E ratio.
n_stocks = 30
best_stocks = PE_values.sort_values(by='P/E ratio')[0:n_stocks]

best_stocks

,P/E ratio
Stocks,
CG,9.12
HOLX,9.86
PFG,10.55
DISH,10.81
CSGP,15.21
DLTR,16.00
REGN,16.71
TROW,17.44
EBAY,17.48


In [57]:
### Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):

    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    
    return closes

In [73]:
### Get the final tickers names to get.
tickers_to_get = best_stocks.index.values.tolist()

### Use our function to get the closes.
get_adj_closes('AMZN', start_date='2018-01-01', end_date='2021-01-01')

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/AMZN/history?period1=1514800800&period2=1609581599&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n

In [75]:
web.get_data_yahoo('AMZN')

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/AMZN/history?period1=1469264400&period2=1627030799&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n